<a href="https://colab.research.google.com/github/mfarhan-113/Parallel-Image-Segmentation/blob/main/Parallel%20image%20segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sequence


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import time

# Function for Sequential processing
def kmeans_segment_sequential(image, k=3):
    reshaped_image = image.reshape((-1, 3))

    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(reshaped_image)

    segmented_image = kmeans.cluster_centers_[kmeans.labels_].reshape(image.shape)

    return segmented_image

def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def save_image(image, output_path):
    image = np.uint8(np.clip(image, 0, 255))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, image)

if __name__ == "__main__":
    image_path = '/content/DJI_20241203153440_0210_D.JPG'
    output_path = '/content/segmented_sequential_image.jpg'

    image = load_image(image_path)

    start_time = time.time()

    segmented_image = kmeans_segment_sequential(image, k=3)

    end_time = time.time()
    print(f"Sequential segmentation time: {end_time - start_time:.4f} seconds")

    save_image(segmented_image, output_path)
    print("Segmentation completed and saved.")


Sequential segmentation time: 17.3716 seconds
Segmentation completed and saved.


#Parallel


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import time
import threading

# Function to apply k-means clustering to a part of the image (parallel)
def kmeans_segment_part(image_part, k=3):
    reshaped_part = image_part.reshape((-1, 3))

    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(reshaped_part)

    segmented_part = kmeans.cluster_centers_[kmeans.labels_].reshape(image_part.shape)

    return segmented_part

# Function to load and preprocess the image
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    return image

# Function to save the segmented image
def save_image(image, output_path):
    image = np.uint8(np.clip(image, 0, 255))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, image)

# Function to divide the image into parts for parallel processing
def divide_image(image, num_parts=2):
    height, width, _ = image.shape
    parts = []
    part_height = height // num_parts
    for i in range(num_parts):
        start_y = i * part_height
        end_y = (i + 1) * part_height if i < num_parts - 1 else height
        parts.append(image[start_y:end_y, :, :])
    return parts

# Function for parallel processing
def parallel_kmeans(image, num_threads=2, k=3):
    parts = divide_image(image, num_parts=num_threads)
    threads = []
    segmented_parts = [None] * num_threads

    def worker(thread_id, image_part):
        segmented_parts[thread_id] = kmeans_segment_part(image_part, k)

    # Start threads
    for i in range(num_threads):
        thread = threading.Thread(target=worker, args=(i, parts[i]))
        threads.append(thread)
        thread.start()

    # Join threads
    for thread in threads:
        thread.join()

    # Combine the segmented parts back into one image
    segmented_image = np.vstack(segmented_parts)

    return segmented_image

# Main function for parallel processing
if __name__ == "__main__":
    image_path = '/content/DJI_20241203153440_0210_D.JPG'
    output_path = '/content/segmented_parallel_image.jpg'

    image = load_image(image_path)

    # Measure time for parallel processing
    start_time = time.time()

    # Perform k-means segmentation (parallel)
    segmented_image = parallel_kmeans(image, num_threads=4, k=3)

    # Measure elapsed time
    end_time = time.time()
    print(f"Parallel segmentation time: {end_time - start_time:.4f} seconds")

    save_image(segmented_image, output_path)
    print("Segmentation completed and saved.")


Parallel segmentation time: 12.0431 seconds
Segmentation completed and saved.
